In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.text_splitter import  RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS
import pypdf

import os
os.environ["OPENAI_API_KEY"] = ""

In [11]:
file_path = "C:\\Users\\James Lee\\Desktop\\HM-Group-Annual-and-Sustainability-Report-2022.pdf"
pdf_reader = pypdf.PdfReader(file_path)

In [20]:
pdfs = pdf_reader.pages[6:8]
text = ''
for i in range(len(pdfs)):
    text = pdfs[i].extract_text()
    text += text
text = text.replace("\n", "")
text

'signed long-term virtual power purchase agreements in the UK, Sweden and Spain to cover electricity consumption in our operations in a majority of European markets. This will not only help us reduce our greenhouse gas emissions, but also secure our energy prices. Our sustainability work was recognised by, among others, the Dow Jones Sustainability World Index for the 11th consecutive year. As one of only 12 global retail companies in the index, we are assessed as a leader in environmental, social and governance performance. Investments in sustainability provide the group with long-term business opportunities. By building strategic partnerships with key stakeholders and growing in various innovative ways such as circular business models, we can grow our business in a way that decouples our financial growth and profitability from the use of finite natural resources. A good example of this is majority-owned fast-growing Sellpy, which is already one of the biggest players in second-hand f

In [21]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 512,
    chunk_overlap  = 32,
    length_function = len,
)
texts = text_splitter.split_text(text)
texts

['signed long-term virtual power purchase agreements in the UK, Sweden and Spain to cover electricity consumption in our operations in a majority of European markets. This will not only help us reduce our greenhouse gas emissions, but also secure our energy prices. Our sustainability work was recognised by, among others, the Dow Jones Sustainability World Index for the 11th consecutive year. As one of only 12 global retail companies in the index, we are assessed as a leader in environmental, social and',
 'in environmental, social and governance performance. Investments in sustainability provide the group with long-term business opportunities. By building strategic partnerships with key stakeholders and growing in various innovative ways such as circular business models, we can grow our business in a way that decouples our financial growth and profitability from the use of finite natural resources. A good example of this is majority-owned fast-growing Sellpy, which is already one of th

In [23]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [25]:
vectorstore = Chroma.from_texts(texts, embeddings)

No embedding_function provided, using default embedding function: DefaultEmbeddingFunction https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2


In [ ]:
qa = ConversationalRetrievalChain.from_llm(ChatOpenAI(temperature=0), vectorstore.as_retriever())
chat_history = []
while True:
    query = input('\nQ: ') 
    if not query:
        break
    result = qa({"question": query + ' (用繁體中文回答)', "chat_history": chat_history})
    print('A:', result['answer'])
    chat_history.append((query, result['answer']))

C:\Users\James Lee\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(



Q: 這篇文章的重點是甚麼?


C:\Users\James Lee\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


A: 這篇文章的重點是H&M集團致力於投資新的商業模式、材料和技術，以推動產品的製造和再製造方式出現根本性轉變，並提供顧客獨特的購物體驗。同時，他們也致力於提高透明度水平，讓顧客能夠更明智地選擇他們購買的產品。公司的目標是在時尚領域持續投資和發展，提供獨特的顧客服務和購物體驗，同時改善客戶體驗，推動回收和更可持續來源材料的規模化和商業化。
